Source: https://stackoverflow.com/questions/7048839/sql-server-query-to-find-all-permissions-access-for-all-users-in-a-database  
  
  
Security Audit Report  
1) List all access provisioned to a sql user or windows user/group directly   
2) List all access provisioned to a sql user or windows user/group through a database or application role  
3) List all access provisioned to the public role  
  
Columns Returned:  
UserName        : SQL or Windows/Active Directory user account.  This could also be an Active Directory group.  
UserType        : Value will be either 'SQL User' or 'Windows User'.  This reflects the type of user defined for the   
                  SQL Server user account.  
DatabaseUserName: Name of the associated user as defined in the database user account.  The database user may not be the  
                  same as the server user.  
Role            : The role name.  This will be null if the associated permissions to the object are defined at directly  
                  on the user account, otherwise this will be the name of the role that the user is a member of.  
PermissionType  : Type of permissions the user/role has on an object. Examples could include CONNECT, EXECUTE, SELECT  
                  DELETE, INSERT, ALTER, CONTROL, TAKE OWNERSHIP, VIEW DEFINITION, etc.  
                  This value may not be populated for all roles.  Some built in roles have implicit permission  
                  definitions.  
PermissionState : Reflects the state of the permission type, examples could include GRANT, DENY, etc.  
                  This value may not be populated for all roles.  Some built in roles have implicit permission  
                  definitions.  
ObjectType      : Type of object the user/role is assigned permissions on.  Examples could include USER_TABLE,   
                  SQL_SCALAR_FUNCTION, SQL_INLINE_TABLE_VALUED_FUNCTION, SQL_STORED_PROCEDURE, VIEW, etc.     
                  This value may not be populated for all roles.  Some built in roles have implicit permission  
                  definitions.            
ObjectName      : Name of the object that the user/role is assigned permissions on.    
                  This value may not be populated for all roles.  Some built in roles have implicit permission  
                  definitions.  
ColumnName      : Name of the column of the object that the user/role is assigned permissions on. This value  
                  is only populated if the object is a table, view or a table value function.

List all access provisioned to a sql user or windows user/group directly

In [9]:
SELECT  SERVERPROPERTY('ServerName') AS "ServerName"
      , [UserName]                   = CASE princ.[type]
                                            WHEN 'S' THEN
                                                princ.[name]
                                       END
      , [UserType]                   = CASE princ.[type]
                                            WHEN 'S' THEN
                                                'SQL User'
                                            WHEN 'U' THEN
                                                'Windows User'
                                       END
      , [DatabaseUserName]           = princ.[name]
      , [Role]                       = NULL
      , [PermissionType]             = perm.[permission_name]
      , [PermissionState]            = perm.[state_desc]
      , [ObjectType]                 = obj.type_desc    --perm.[class_desc],       
      , [ObjectName]                 = OBJECT_NAME(perm.major_id)
      , [ColumnName]                 = col.[name]
      , sp.is_disabled
      , GETDATE()                    AS [ReportRunOn]
FROM
    --database user
    sys.database_principals         princ
    LEFT JOIN
    --Permissions
    sys.database_permissions        perm
        ON perm.[grantee_principal_id] = princ.[principal_id]
    LEFT JOIN
    --Table columns
    sys.columns                     col
        ON col.[object_id]             = perm.major_id
           AND  col.[column_id]        = perm.[minor_id]
    LEFT JOIN sys.objects           obj
        ON perm.[major_id]             = obj.[object_id]
    LEFT JOIN sys.server_principals sp
        ON princ.name                  = sp.name
WHERE   princ.[type] IN ( 'S', 'U' )
UNION
--List all access provisioned to a sql user or windows user/group through a database or application role
SELECT  SERVERPROPERTY('ServerName') AS "ServerName"
      , CASE memberprinc.[type]
             WHEN 'S' THEN
                 memberprinc.[name]
        END                          AS UserName
      , [UserType]                   = CASE memberprinc.[type]
                                            WHEN 'S' THEN
                                                'SQL User'
                                            WHEN 'U' THEN
                                                'Windows User'
                                       END
      , [DatabaseUserName]           = memberprinc.[name]
      , [Role]                       = roleprinc.[name]
      , [PermissionType]             = perm.[permission_name]
      , [PermissionState]            = perm.[state_desc]
      , [ObjectType]                 = obj.type_desc    --perm.[class_desc],   
      , [ObjectName]                 = OBJECT_NAME(perm.major_id)
      , [ColumnName]                 = col.[name]
      , sp.is_disabled
      , GETDATE()                    AS [log_date_time]
FROM
    --Role/member associations
    sys.database_role_members       members
    JOIN
    --Roles
    sys.database_principals         roleprinc
        ON roleprinc.[principal_id]    = members.[role_principal_id]
    JOIN
    --Role members (database users)
    sys.database_principals         memberprinc
        ON memberprinc.[principal_id]  = members.[member_principal_id]
    LEFT JOIN
    --Permissions
    sys.database_permissions        perm
        ON perm.[grantee_principal_id] = roleprinc.[principal_id]
    LEFT JOIN
    --Table columns
    sys.columns                     col
        ON col.[object_id]             = perm.major_id
           AND  col.[column_id]        = perm.[minor_id]
    LEFT JOIN sys.objects           obj
        ON perm.[major_id]             = obj.[object_id]
    LEFT JOIN sys.server_principals sp
        ON memberprinc.[name]          = sp.name
UNION
--List all access provisioned to the public role, which everyone gets by default
SELECT  SERVERPROPERTY('ServerName') AS "ServerName"
      , [UserName]                   = '{All Users}'
      , [UserType]                   = '{All Users}'
      , [DatabaseUserName]           = '{All Users}'
      , [Role]                       = roleprinc.[name]
      , [PermissionType]             = perm.[permission_name]
      , [PermissionState]            = perm.[state_desc]
      , [ObjectType]                 = obj.type_desc    --perm.[class_desc],  
      , [ObjectName]                 = OBJECT_NAME(perm.major_id)
      , [ColumnName]                 = col.[name]
      , sp.is_disabled
      , GETDATE()                    AS ReportDate
FROM
    --Roles
    sys.database_principals         roleprinc
    LEFT JOIN
    --Role permissions
    sys.database_permissions        perm
        ON perm.[grantee_principal_id] = roleprinc.[principal_id]
    LEFT JOIN
    --Table columns
    sys.columns                     col
        ON col.[object_id]             = perm.major_id
           AND  col.[column_id]        = perm.[minor_id]
    LEFT JOIN sys.server_principals sp
        ON roleprinc.name              = sp.name
    JOIN
    --All objects   
    sys.objects                     obj
        ON obj.[object_id]             = perm.[major_id]
WHERE
    --Only roles
    roleprinc.[type]  = 'R'
    AND
    --Only public role
    roleprinc.[name]  = 'public'
    AND
    --Only objects of ours, not the MS objects
    obj.is_ms_shipped = 0
ORDER BY princ.[Name]
       , OBJECT_NAME(perm.major_id)
       , col.[name]
       , perm.[permission_name]
       , perm.[state_desc]
       , obj.type_desc  --perm.[class_desc] 


;


SELECT  DISTINCT
        SERVERPROPERTY('ServerName') AS "ServerName"
      , rp.name
      , ObjectType                   = rp.type_desc
      , PermissionType               = pm.class_desc
      , pm.permission_name
      , pm.state_desc
      , ObjectType                   = CASE
                                            WHEN obj.type_desc IS NULL
                                                 OR   obj.type_desc = 'SYSTEM_TABLE' THEN
                                                pm.class_desc
                                            ELSE
                                                obj.type_desc
                                       END
      , s.Name                       AS SchemaName
      , [ObjectName]                 = ISNULL(ss.name, OBJECT_NAME(pm.major_id))
FROM    sys.database_principals            rp
    INNER JOIN sys.database_permissions pm
        ON pm.grantee_principal_id = rp.principal_id
    LEFT JOIN sys.schemas               ss
        ON pm.major_id             = ss.schema_id
    LEFT JOIN sys.objects               obj
        ON pm.[major_id]           = obj.[object_id]
    LEFT JOIN sys.schemas               s
        ON s.schema_id             = obj.schema_id
WHERE   rp.type_desc      = 'DATABASE_ROLE'
        AND pm.class_desc <> 'DATABASE'
ORDER BY rp.name
       , rp.type_desc
       , pm.class_desc;


(68 rows affected)

(217 rows affected)

Total execution time: 00:00:00.334

ServerName,UserName,UserType,DatabaseUserName,Role,PermissionType,PermissionState,ObjectType,ObjectName,ColumnName,is_disabled,ReportRunOn
qfc4q6krnd,alexa,SQL User,alexa,db_datareader,NULL,NULL,NULL,NULL,NULL,NULL,2021-12-03 18:10:50.650
qfc4q6krnd,alexa,SQL User,alexa,NULL,CONNECT,GRANT,NULL,NULL,NULL,NULL,2021-12-03 18:10:50.650
qfc4q6krnd,bconklin,SQL User,bconklin,db_datareader,NULL,NULL,NULL,NULL,NULL,NULL,2021-12-03 18:10:50.650
qfc4q6krnd,bconklin,SQL User,bconklin,NULL,CONNECT,GRANT,NULL,NULL,NULL,NULL,2021-12-03 18:10:50.650
qfc4q6krnd,briant,SQL User,briant,db_datareader,NULL,NULL,NULL,NULL,NULL,NULL,2021-12-03 18:10:50.650
qfc4q6krnd,briant,SQL User,briant,NULL,CONNECT,GRANT,NULL,NULL,NULL,NULL,2021-12-03 18:10:50.650
qfc4q6krnd,cbeyne,SQL User,cbeyne,db_datareader,NULL,NULL,NULL,NULL,NULL,NULL,2021-12-03 18:10:50.650
qfc4q6krnd,cbeyne,SQL User,cbeyne,NULL,CONNECT,GRANT,NULL,NULL,NULL,NULL,2021-12-03 18:10:50.650
qfc4q6krnd,chrisr,SQL User,chrisr,db_datareader,NULL,NULL,NULL,NULL,NULL,NULL,2021-12-03 18:10:50.650
qfc4q6krnd,chrisr,SQL User,chrisr,NULL,CONNECT,GRANT,NULL,NULL,NULL,NULL,2021-12-03 18:10:50.650


ServerName,name,ObjectType,PermissionType,permission_name,state_desc,ObjectType,SchemaName,ObjectName
qfc4q6krnd,public,DATABASE_ROLE,OBJECT_OR_COLUMN,SELECT,GRANT,OBJECT_OR_COLUMN,NULL,all_columns
qfc4q6krnd,public,DATABASE_ROLE,OBJECT_OR_COLUMN,SELECT,GRANT,OBJECT_OR_COLUMN,NULL,all_objects
qfc4q6krnd,public,DATABASE_ROLE,OBJECT_OR_COLUMN,SELECT,GRANT,OBJECT_OR_COLUMN,NULL,all_parameters
qfc4q6krnd,public,DATABASE_ROLE,OBJECT_OR_COLUMN,SELECT,GRANT,OBJECT_OR_COLUMN,NULL,all_sql_modules
qfc4q6krnd,public,DATABASE_ROLE,OBJECT_OR_COLUMN,SELECT,GRANT,OBJECT_OR_COLUMN,NULL,all_views
qfc4q6krnd,public,DATABASE_ROLE,OBJECT_OR_COLUMN,SELECT,GRANT,OBJECT_OR_COLUMN,NULL,allocation_units
qfc4q6krnd,public,DATABASE_ROLE,OBJECT_OR_COLUMN,SELECT,GRANT,OBJECT_OR_COLUMN,NULL,assemblies
qfc4q6krnd,public,DATABASE_ROLE,OBJECT_OR_COLUMN,SELECT,GRANT,OBJECT_OR_COLUMN,NULL,assembly_files
qfc4q6krnd,public,DATABASE_ROLE,OBJECT_OR_COLUMN,SELECT,GRANT,OBJECT_OR_COLUMN,NULL,assembly_modules
qfc4q6krnd,public,DATABASE_ROLE,OBJECT_OR_COLUMN,SELECT,GRANT,OBJECT_OR_COLUMN,NULL,assembly_references


In [2]:
SELECT  DP1.name                       AS DatabaseRoleName
      , ISNULL(DP2.name, 'No members') AS DatabaseUserName
FROM    sys.database_role_members               AS DRM
    RIGHT OUTER JOIN sys.database_principals AS DP1
        ON DRM.role_principal_id   = DP1.principal_id
    LEFT OUTER JOIN sys.database_principals  AS DP2
        ON DRM.member_principal_id = DP2.principal_id
WHERE   DP1.type = 'R'
ORDER BY DP1.name;

(25 rows affected)

Total execution time: 00:00:00.111

DatabaseRoleName,DatabaseUserName
db_accessadmin,No members
db_backupoperator,No members
db_datareader,skimminau
db_datareader,jroche
db_datareader,cbeyne
db_datareader,masong
db_datareader,alexa
db_datareader,briant
db_datareader,colterl
db_datareader,jamesr


In [ ]:
--This misses connect grants maybe not as good as others
;WITH [explicit]
   AS (   SELECT    [p].[principal_id]
                  , [p].[name]
                  , [p].[type_desc]
                  , [p].[create_date]
                  , [dbp].[permission_name] COLLATE SQL_Latin1_General_CP1_CI_AS [permission]
                  , CAST('' AS SYSNAME)                                          [grant_through]
          FROM  [sys].[database_permissions]         [dbp]
              INNER JOIN [sys].[database_principals] [p]
                  ON [dbp].[grantee_principal_id] = [p].[principal_id]
          WHERE (
                    [dbp].[type] IN ( 'IN', 'UP', 'DL', 'CL', 'DABO', 'IM', 'SL', 'TO' )
                    OR  [dbp].[type] LIKE 'AL%'
                    OR  [dbp].[type] LIKE 'CR%'
                )
                AND [dbp].[state] IN ( 'G', 'W' )
          UNION ALL
          SELECT    [dp].[principal_id]
                  , [dp].[name]
                  , [dp].[type_desc]
                  , [dp].[create_date]
                  , [p].[permission]
                  , [p].[name] [grant_through]
          FROM  [sys].[database_principals]            [dp]
              INNER JOIN [sys].[database_role_members] [rm]
                  ON [rm].[member_principal_id] = [dp].[principal_id]
              INNER JOIN [explicit]                    [p]
                  ON [p].[principal_id]         = [rm].[role_principal_id])
    , [fixed]
   AS (   SELECT    [dp].[principal_id]
                  , [dp].[name]
                  , [dp].[type_desc]
                  , [dp].[create_date]
                  , [p].[name]          [permission]
                  , CAST('' AS SYSNAME) [grant_through]
          FROM  [sys].[database_principals]            [dp]
              INNER JOIN [sys].[database_role_members] [rm]
                  ON [rm].[member_principal_id] = [dp].[principal_id]
              INNER JOIN [sys].[database_principals]   [p]
                  ON [p].[principal_id]         = [rm].[role_principal_id]
          WHERE [p].[name] IN ( 'db_owner'
                              , 'db_datareader'
                              , 'db_datawriter'
                              , 'db_ddladmin'
                              , 'db_securityadmin'
                              , 'db_accessadmin'
                              , 'dbmanager'
                              , 'loginmanager'
                              )
          UNION ALL
          SELECT    [dp].[principal_id]
                  , [dp].[name]
                  , [dp].[type_desc]
                  , [dp].[create_date]
                  , [p].[permission]
                  , [p].[name] [grant_through]
          FROM  [sys].[database_principals]            [dp]
              INNER JOIN [sys].[database_role_members] [rm]
                  ON [rm].[member_principal_id] = [dp].[principal_id]
              INNER JOIN [fixed]                       [p]
                  ON [p].[principal_id]         = [rm].[role_principal_id])
SELECT  DISTINCT
        SERVERPROPERTY('ServerName') AS "ServerName"
      , DB_NAME()      AS 'database'
      , [name]         AS username
      , [type_desc]
      , [create_date]
      , [permission]
      , [grant_through]
      , GETDATE()      AS ReportRun
FROM    [explicit]
WHERE   [type_desc] NOT IN ( 'DATABASE_ROLE' )
        AND [name] = 'jroche'
UNION ALL
SELECT  DISTINCT
        SERVERPROPERTY('ServerName') AS "ServerName"
      , DB_NAME()
      , [name]
      , [type_desc]
      , [create_date]
      , [permission]
      , [grant_through]
      , GETDATE() AS "GetDate"
FROM    [fixed]
WHERE   [type_desc] NOT IN ( 'DATABASE_ROLE' )
        AND name = 'jroche'
ORDER BY 1
       , 2
OPTION (MAXRECURSION 10);
